In [72]:
import pandas as pd
import ipysheet
import ipywidgets as widgets
from IPython.display import HTML
import warnings
warnings.simplefilter('ignore')

%matplotlib notebook

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Нажмите чтобы отобразить/скрыть код."></form>''')



# Задача о распределении средств
<br>&#8195;&#8195;Метод динамического программирования дает возможность находить последовательные оптимальные решения в задачах, разделенных на этапы.
    
<br>&#8195;&#8195;Изложим схему применения этого метода на следующей модели. Рассмотрим некоторую управляемую систему, которая может находиться в одном из нескольких состояний. На каждом этапе в результате применения управляющего воздействия (управления) система может изменить свое состояние или остаться в прежнем состоянии. Эффективность процесса управления характеризуется целевой функцией прибыли, зависящей от состояния системы и применяемого управления.
    
<br>&#8195;&#8195;0 этап. В начальный момент времени система находится в исходном состоянии $x_0$.
    
<br>&#8195;&#8195;1 этап. В результате применения управления $y_1$ система переходит из состояния $x_0$ в состояние<br>
            <center>$x_1$=$g_1$($x_0$,$y_1$)</center>при этом получается прибыль<br><center>$h_1$($x_0$,$y_1$)</center><br>
        
<br>&#8195;&#8195;2 этап. В результате применения управления $y_2$ система переходит из состояния $x_1$ в состояние<br>
            <center>$x_2$=$g_2$($x_1$,$y_2$)</center><br>
    при этом получается прибыль<br>
            <center>$h_2$($x_1$,$y_2$)</center><br>
    и так далее.
    
<br>&#8195;&#8195;За N этапов получается последовательность состояний $x_0$, $x_1$, $x_2$, ..., $x_n$ и последовательность управлений $y_1$, $y_2$, ..., $y_n$, где<br>
            <center>$x_{n+1}$=$g_n$($x_n$,$y_{n+1}$), ${\qquad}$ n=${\overline{0,N}}$</center><br>
    а общая прибыль на каждом этапе вычисляется по формуле<br>
            <center>$J_n$($x_0$, $y_1$, ...,$y_n$)=$h_1$($x_0$, $y_1$)+$h_2$($x_1$, $y_2$)+...+$h_n$($x_{n-1}$, $y_n$), ${\qquad}$ n=${\overline{1,N}}$</center><br>
    
<br>&#8195;&#8195;Нашей целью является отыскание такой последовательности оптимальных управлений \{$y_1^*$, $y_2^*$, ..., $y_N^*$\},чтобы функция прибыли $J_n$ достигла максимума<br>
            <center>$J_n$($x_0$, $y_1^*$, ..., $y_N^*$)=max $J_n$($x_0$, $y_1$, ...,$y_n$)</center>
<br>&#8195;&#8195;Принцип оптимальности Беллмана утверждает, что на последовательности оптимальных управлений \{$y_1^*$, $y_2^*$, ..., $y_N^*$\} должна достигать максимума каждая из функций<br>
            <center>$f_n$($x_{n-1}$, $y_n$, $y_n
            {n+1}$, ..., $y_N$)=$h_n$($x_{n-1}$, $y_n$)+$h_{n+1}$($x_n$, $y_{n+1}$)+...+$h_N$($x_{N-1}$, $y_N$), ${\qquad}$ n=${\overline{1,N}}$</center>
<br>&#8195;&#8195;Если ввести обозначения<br>
            <center>$\varphi_n(x_{n-1})$=max $f_n$($x_{n-1}$, $y_n$, $y_n
            {n+1}$, ..., $y_N$) ${\qquad}$ n=${\overline{1,N}}$</center>
<br>то из принципа оптимальности Беллмана вытекает, что функции $\varphi_n(x_{n-1})$ должны удовлетворять следующим функциональным уравнениям Беллмана:<br>
            <center>$\varphi_n(x_{n-1})$=max[ $\varphi_{n+1}(g_n(x_{n-1},y_n))+h_n(x_{n-1},y_n)$] ${\qquad}$ n=${\overline{1,N}}$</center>
    
<br>&#8195;&#8195;Решение уравнений Беллмана позволяет найти последовательность оптимальных управлений и оптимальное значение функции прибыли.
    
<br>&#8195;&#8195;Замечание. Для того, чтобы методом динамического программирования находить не максимум функции прибыли, как было изложено выше, а минимум функции затрат достаточно лишь заменить во всех соответствующих формулах настоящего параграфа максимум на минимум.

## Пример решения задачи о распределении средств
<br>&#8195;&#8195;Составить план распределения суммы в 6 миллионов долларов между четырьмя предприятиями $\text{П}_1$, $\text{П}_2$, $\text{П}_3$,$\text{П}_4$ приносящий наибольшую прибыль, если в каждое из предприятий может быть вложено 1 - 6 миллионов долларов, а прибыль каждого из предприятий задана в Таблице:

In [73]:
table = [[0, 0, 0, 0],
[2.0, 2.1, 1.7, 1.9],
[2.5, 2.9, 2.4, 2.3],
[3.8, 3.6, 3.7, 3.6],
[4.8, 4.7, 4.4, 4.3],
[5.8, 5.4, 5.7, 5.9],
[6.3, 6.5, 6.6, 6.4]]
check = 0
def finder(u,n):
    for i in range(len(tmp[n])):
        if((tmp[n][i][0]==u)and(tmp[n][i][4]!=0)):
            return tmp[n][i][4]
    return 0

def finder2(u,n):
    for i in range(len(tmp[n])):
        if((tmp[n][i][0]==u)and(tmp[n][i][6]!=0)):
            return tmp[n][i][6]
    return 0

def finder3(u,n):
    for i in range(len(tmp[n])):
        if((tmp[n][i][0]==u)and(tmp[n][i][6]!=0)):
            return tmp[n][i][7]
    return 0

In [74]:
%%HTML
<style type="text/css">
.mystyle table, td, th {
    border: 1px  black solid !important;
    width: 150px !important;
    color: black !important;
    index: False !important;
    font-family:monospace !important;
    font-size: 14px !important;
    
}

</style>

In [75]:
#background-color: #fdbfca !important;

df = pd.DataFrame(table,columns=['$Пр_1$','$Пр_2$','$Пр_3$','$Пр_4$'])
sum = 0
for i in range(len(table)+1):
    sum+=i
n = 4 
list_a = [0,1]
i = 2
while(i<sum-list_a[-1]):
    list_a.append(i+list_a[-1])
    i=i+1
tmp = [[[] for i in range(sum)] for j in range(n)]
df.index.name = 'Размер инвестиций'
df

,$Пр_1$,$Пр_2$,$Пр_3$,$Пр_4$
Размер инвестиций,,,,
0,0.0,0.0,0.0,0.0
1,2.0,2.1,1.7,1.9
2,2.5,2.9,2.4,2.3
3,3.8,3.6,3.7,3.6
4,4.8,4.7,4.4,4.3
5,5.8,5.4,5.7,5.9
6,6.3,6.5,6.6,6.4


<br>&#8195;&#8195;Разобьем процесс выделения средств предприятиям на 4 этапа: на первом этапе выделяется $y_1$ средств предприятию $\text{П}_1$, на втором – $y_2$ средств предприятию $\text{П}_2$, на третьем – $y_3$ средств предприятию $\text{П}_3$,на четвертом – $y_4$ средств предприятию $\text{П}_4$.
    
<br>&#8195;&#8195;Будем считать состоянием системы $x_i$(i=0,1,2,3,4) ту сумму средств, которая осталась нераспределенной после i-го этапа. Поскольку необходимо распределить все 4 миллиона долларов, то $x_0$=6. Тогда<br>    
           <center>$x_n$=$x_{n-1}$-$y_n$, ${\qquad}$ n=1,2,3,4.</center><br>

Поясним построение таблиц и последовательность проведения расчетов.
Столбцы 1 (вложенные средства), 2 (проект) и 3 (остаток средств) для всех трех таблиц одинаковы, поэтому их можно было бы сделать общими. Столбец 4 заполняется на основе исходных данных о функциях дохода, значения в столбце 5 берутся из столбца 7 предыдущей таблицы, столбец 6 заполняется суммой значений столбцов 4 и 5 (в таблице 4-го шага столбцы 5 и 6 отсутствуют).
В столбце 7 записывается максимальное значение предыдущего столбца для фиксированного начального состояния, и в 8 столбце записывается управление из 2 столбца, на котором достигается максимум в 7.

Строки с максимальным значением для этого размера вложений выделены розовым цветом. Каждое увелечение размера вложений выделено серым цветом.

Предположим, что все средства в количестве $x_4$ = 6 отданы предприятию №4. В этом случае, максимальный доход, как это видно из таблицы, составит $f_4(u_4)$ = 6.4, следовательно, $F_4(e_4) = f_4(u_4)$

In [76]:
print("Шаг 1")
fu = table[0][n-1]
en = 1
u = 0
en1 = 1
fe = 0
un = 0

t = 0
for i in range(len(table)):
    for u in range(i+1):
        fu = table[u][n-1]
        fe = fu
        tmp[n-1][t].append(i)
        tmp[n-1][t].append(u)
        tmp[n-1][t].append(i-u)
        tmp[n-1][t].append(fu)
        if(tmp[n-1][t][2] == 0):
            tmp[n-1][t].append(fe)
            tmp[n-1][t].append(u)
        else:
            tmp[n-1][t].append(0)
            tmp[n-1][t].append(0)
        t +=1
list_b = []
for i in range(sum):
    if(tmp[3][i][5]>0):
        list_b.append(i)
df = pd.DataFrame(tmp[n-1],columns=['$e_3$','$u_4$','$e_4 = e_3 - u_4$','$f_4(u_4)$','$F^*_4(e_4)$','$u_4(e_4)$'])
df = df.style.apply(lambda x: ['background: #D3D3D3' if x.name in list_a else '' for i in x],axis=1).apply(lambda x: ['background: #fdbfca' if x.name in list_b else '' for i in x],axis=1).set_properties(**{'width': '150px','font-size': '14pt', 'font-family': 'monospace','border-collapse': 'collapse','border': '1px solid black'}).hide_index()
df

Шаг 1


$e_3$,$u_4$,$e_4 = e_3 - u_4$,$f_4(u_4)$,$F^*_4(e_4)$,$u_4(e_4)$
0,0,0,0,0,0
1,0,1,0,0,0
1,1,0,1.9,1.9,1
2,0,2,0,0,0
2,1,1,1.9,0,0
2,2,0,2.3,2.3,2
3,0,3,0,0,0
3,1,2,1.9,0,0
3,2,1,2.3,0,0
3,3,0,3.6,3.6,3


2-ой шаг. k = 3.
Определяем оптимальную стратегию при распределении денежных средств между предприятиями №3, 4. При этом рекуррентное соотношение Беллмана имеет вид: $F_3(e_3)$ = max($x_3 ≤ e_3)(f_3(u_3) + F_4(e_3-u_3))$

In [77]:
n = 3 
fu = 0
en = 1
u = 0
en1 = 1
fe = 0
un = 0
t = 0
for i in range(len(table)):
    for u in range(i+1):
        fu = table[u][n-1]
        fe = fu
        tmp[n-1][t].append(i)
        tmp[n-1][t].append(u)
        tmp[n-1][t].append(i-u)
        tmp[n-1][t].append(fu)
        tmp[n-1][t].append(finder(i-u,n))
        tmp[n-1][t].append(finder(i-u,n)+fu)
        t +=1
    maxx = tmp[n-1][t-1][5]
    im = t-1
    for x in range(t-1-u,t-1):
        if(tmp[n-1][x][5]>maxx):
            maxx=tmp[n-1][x][5]
            im = x
    tmp[n-1][im].append(maxx)
    tmp[n-1][im].append(tmp[n-1][im][1])
    for x in range(t-1-u,t):
        if(x!=im):
            tmp[n-1][x].append(0)
            tmp[n-1][x].append(0)
list_b = []
for i in range(sum):
    if(tmp[2][i][7]>0):
        list_b.append(i)
df = pd.DataFrame(tmp[n-1],columns=['$e_2$','$u_3$','$e_3 = e_2 - u_3$','$f_3(u_3)$','$F^*_3(e_2)$','$F_2(u_3,e_2)$','$F^*_3(e_3)$','$u_3(e_3)$'])
df = df.style.apply(lambda x: ['background: #D3D3D3' if x.name in list_a else '' for i in x],axis=1).apply(lambda x: ['background: #fdbfca' if x.name in list_b else '' for i in x],axis=1).set_properties(**{'width': '150px','font-size': '14pt', 'font-family': 'monospace','border-collapse': 'collapse','border': '1px solid black'}).hide_index()
df

$e_2$,$u_3$,$e_3 = e_2 - u_3$,$f_3(u_3)$,$F^*_3(e_2)$,"$F_2(u_3,e_2)$",$F^*_3(e_3)$,$u_3(e_3)$
0,0,0,0,0,0,0,0
1,0,1,0,1.9,1.9,1.9,0
1,1,0,1.7,0,1.7,0,0
2,0,2,0,2.3,2.3,0,0
2,1,1,1.7,1.9,3.6,3.6,1
2,2,0,2.4,0,2.4,0,0
3,0,3,0,3.6,3.6,0,0
3,1,2,1.7,2.3,4,0,0
3,2,1,2.4,1.9,4.3,4.3,2
3,3,0,3.7,0,3.7,0,0


3-ий шаг. k = 2.
Определяем оптимальную стратегию при распределении денежных средств между предприятиями №2, 3, 4. При этом рекуррентное соотношение Беллмана имеет вид: $F_2(e_2) = max(x_2 ≤ e_2)(f_2(u_2) + F_3(e_2-u_2))$

In [78]:

n = 2
fu = 0
en = 1
u = 0
en1 = 1
fe = 0
un = 0
t = 0
for i in range(len(table)):
    for u in range(i+1):
        fu = table[u][n-1]
        fe = fu
        tmp[n-1][t].append(i)
        tmp[n-1][t].append(u)
        tmp[n-1][t].append(i-u)
        tmp[n-1][t].append(fu)
        tmp[n-1][t].append(finder2(i-u,n))
        tmp[n-1][t].append(finder2(i-u,n)+fu)
        t +=1
    maxx = tmp[n-1][t-1][5]
    im = t-1
    for x in range(t-1-u,t-1):
        if(tmp[n-1][x][5]>maxx):
            maxx=tmp[n-1][x][5]
            im = x
    tmp[n-1][im].append(maxx)
    tmp[n-1][im].append(tmp[n-1][im][1])
    for x in range(t-1-u,t):
        if(x!=im):
            tmp[n-1][x].append(0)
            tmp[n-1][x].append(0)
list_b = []
for i in range(sum):
    if(tmp[1][i][7]>0):
        list_b.append(i)
df = pd.DataFrame(tmp[n-1],columns=['$e_1$','$u_2$','$e_2 = e_1 - u_2$','$f_2(u_2)$','$F^*_2(e_1)$','$F_1(u_2,e_1)$','$F^*_2(e_2)$','$u_2(e_2)$'])
df = df.style.apply(lambda x: ['background: #D3D3D3' if x.name in list_a else '' for i in x],axis=1).apply(lambda x: ['background: #fdbfca' if x.name in list_b else '' for i in x],axis=1).set_properties(**{'width': '150px','font-size': '14pt', 'font-family': 'monospace','border-collapse': 'collapse','border': '1px solid black'}).hide_index()
df

$e_1$,$u_2$,$e_2 = e_1 - u_2$,$f_2(u_2)$,$F^*_2(e_1)$,"$F_1(u_2,e_1)$",$F^*_2(e_2)$,$u_2(e_2)$
0,0,0,0,0,0,0,0
1,0,1,0,1.9,1.9,0,0
1,1,0,2.1,0,2.1,2.1,1
2,0,2,0,3.6,3.6,0,0
2,1,1,2.1,1.9,4,4,1
2,2,0,2.9,0,2.9,0,0
3,0,3,0,4.3,4.3,0,0
3,1,2,2.1,3.6,5.7,5.7,1
3,2,1,2.9,1.9,4.8,0,0
3,3,0,3.6,0,3.6,0,0


4-ый шаг. k = 1.
Определяем оптимальную стратегию при распределении денежных средств между предприятиями №1, 2, 3, 4. При этом рекуррентное соотношение Беллмана имеет вид: $F_1(e_1) = max(x_1 ≤ e_1)(f_1(u_1) + F_2(e_1-u_1))$

In [79]:
n = 1 
fu = 0
en = 1
u = 0
en1 = 1
fe = 0
un = 0
t = 0
for i in range(len(table)):
    for u in range(i+1):
        fu = table[u][n-1]
        fe = fu
        tmp[n-1][t].append(i)
        tmp[n-1][t].append(u)
        tmp[n-1][t].append(i-u)
        tmp[n-1][t].append(fu)
        tmp[n-1][t].append(finder2(i-u,n))
        tmp[n-1][t].append(finder2(i-u,n)+fu)
        t +=1
    maxx = tmp[n-1][t-1][5]
    im = t-1
    for x in range(t-1-u,t-1):
        if(tmp[n-1][x][5]>maxx):
            maxx=tmp[n-1][x][5]
            im = x
    tmp[n-1][im].append(maxx)
    tmp[n-1][im].append(tmp[n-1][im][1])
    for x in range(t-1-u,t):
        if(x!=im):
            tmp[n-1][x].append(0)
            tmp[n-1][x].append(0)
list_b = []
for i in range(sum):
    if(tmp[0][i][7]>0):
        list_b.append(i)
df = pd.DataFrame(tmp[n-1],columns=['$e_0$','$u_1$','$e_1 = e_0 - u_1$','$f_1(u_1)$','$F^*_1(e_0)$','$F_0(u_1,e_0)$','$F^*_1(e_1)$','$u_1(e_1)$'])
df = df.style.apply(lambda x: ['background: #D3D3D3' if x.name in list_a else '' for i in x],axis=1).apply(lambda x: ['background: #fdbfca' if x.name in list_b else '' for i in x],axis=1).set_properties(**{'width': '150px','font-size': '14pt', 'font-family': 'monospace','border-collapse': 'collapse','border': '1px solid black'}).hide_index()
df

$e_0$,$u_1$,$e_1 = e_0 - u_1$,$f_1(u_1)$,$F^*_1(e_0)$,"$F_0(u_1,e_0)$",$F^*_1(e_1)$,$u_1(e_1)$
0,0,0,0,0,0,0,0
1,0,1,0,2.1,2.1,2.1,0
1,1,0,2,0,2,0,0
2,0,2,0,4,4,0,0
2,1,1,2,2.1,4.1,4.1,1
2,2,0,2.5,0,2.5,0,0
3,0,3,0,5.7,5.7,0,0
3,1,2,2,4,6,6,1
3,2,1,2.5,2.1,4.6,0,0
3,3,0,3.8,0,3.8,0,0


Из таблицы 4-го шага имеем $F^*_1(e_0 = 6) = 9.7.$ То есть максимальный доход всей системы при количестве средств $e_0$ = 6 равен 9.7
Из этой же таблицы получаем, что 1-му предприятию следует выделить $u^*_1(e_0 = 6)$ = 1
При этом остаток средств составит:
$e_1 = e_0 - u_1 = 6 - 1 = 5$
Из таблицы 3-го шага имеем $F^*_2(e_1 = 5) = 7.7$. То есть максимальный доход всей системы при количестве средств $e_1$ = 5 равен 7.7
Из этой же таблицы получаем, что 2-му предприятию следует выделить $u^*_2(e_1 = 5) = 1$
При этом остаток средств составит:
$e_2 = e_1 - u_2 = 5 - 1 = 4$
Из таблицы 2-го шага имеем $F^*_3(e_2 = 4) = 5.6.$ То есть максимальный доход всей системы при количестве средств $e_2$ = 4 равен 5.6
Из этой же таблицы получаем, что 3-му предприятию следует выделить $u^*_3(e_2 = 4) = 3$
При этом остаток средств составит:
$e_3 = e_2 - u_3 = 4 - 3 = 1$

In [80]:
print("Первому предприятию: "+str(tmp[n-1][im][7]))
ost = len(table)-1 - tmp[n-1][im][7]
tp = finder3(ost,n)
print("Второму предприятию: "+str(tp))
ost-=tp
tp = finder3(ost,n+1)
print("Третьему предприятию: "+str(tp))
ost-=tp
print("Четвертому предприятию: "+str(ost))

Первому предприятию: 1
Второму предприятию: 1
Третьему предприятию: 3
Четвертому предприятию: 1


In [81]:
q = 0
s = 0
i = 0

In [82]:
from ipywidgets import Button, Layout

quant = widgets.Text(placeholder='Количество предприятий',description='',disabled=False)
summ = widgets.Text(placeholder='Распределяемые средства',description='',disabled=False)
interval = widgets.Text(placeholder='Интервал распределения',description='',disabled=False)

display(quant)
def on_q_clicked():
    global q
    q = int(quant.value)
    if(q>0):
        quant.value = ""
        print("Количество предприятий: "+str(q))
    else:
        print("Число должно быть больше 0")
        
im = widgets.interact_manual(on_q_clicked);

im.widget.children[0].description = 'Добавить'
im.widget.children[0].layout = Layout(width='30%')
display(summ)

def on_s_clicked():
    global s
    s = int(summ.value)
    if(s>0):
        summ.value = ""
        print("Распределяемые средства: "+str(s))
    else:
        print("Число должно быть больше 0")
        
im = widgets.interact_manual(on_s_clicked);

im.widget.children[0].description = 'Добавить'
im.widget.children[0].layout = Layout(width='30%')
display(interval)      
def on_i_clicked():
    global i
    i = int(interval.value)
    if(i>0):
        interval.value = ""
        print("Интервал распределения: "+str(i))
    else:
        print("Число должно быть больше 0")
    
im = widgets.interact_manual(on_i_clicked);

im.widget.children[0].description = 'Добавить'
im.widget.children[0].layout = Layout(width='30%')



Text(value='', placeholder='Количество предприятий')

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

Text(value='', placeholder='Распределяемые средства')

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

Text(value='', placeholder='Интервал распределения')

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [83]:
import ipysheet
from ipysheet import sheet, from_dataframe
from ipywidgets import VBox


col_names = []
row_names = []
for iterator in range(1,q+1):
    col_names.append("Предприятие "+str(iterator))
for iterator in range(0,s+1,i):
    row_names.append(str(iterator))

sheet = ipysheet.sheet(rows=s+1/i, columns=q,column_headers=col_names,row_headers=row_names,layout=Layout(height='auto', width='auto'),row_width='')
for iterator1 in range(q):
    for iterator2 in range(0,s+1,i):
        ipysheet.cell(iterator2,iterator1,0)

sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='numeric', value=0), Cell(column_…

In [85]:
df2 = ipysheet.to_dataframe(sheet)
df2

,Предприятие 1,Предприятие 2,Предприятие 3,Предприятие 4
0,1,32,2,1
1,3,4,0,0
2,0,2,0,0
3,0,0,0,0
4,0,4,5,0
5,0,0,0,5
6,0,0,0,0


In [86]:
col_names =['$e_3$','$u_4$','$e_4 = e_3 - u_4$','$f_4(u_4)$','$F^*_4(e_4)$','$u_4(e_4)$']
row_names = []
for iterator in range(0,(s+1)*q,i):
    row_names.append(str(iterator))

sheet_ans_1 = ipysheet.sheet(rows=(s+1)*q, columns=6,column_headers=col_names,row_headers=row_names,layout=Layout(height='auto', width='auto'),row_width='')
for iterator1 in range(6):
    for iterator2 in range(0,(s+1)*q,i):
        ipysheet.cell(iterator2,iterator1,0)

sheet_ans_1

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='numeric', value=0), Cell(column_…

In [87]:
df3 = ipysheet.to_dataframe(sheet_ans_1)
df3

,$e_3$,$u_4$,$e_4 = e_3 - u_4$,$f_4(u_4)$,$F^*_4(e_4)$,$u_4(e_4)$
0,1,2,3,4,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,0,0,0,0,0,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


In [88]:
col_names =['$e_2$','$u_3$','$e_3 = e_2 - u_3$','$f_3(u_3)$','$F^*_3(e_2)$','$F_2(u_3,e_2)$','$F^*_3(e_3)$','$u_3(e_3)$']
row_names = []
for iterator in range(0,(s+1)*q,i):
    row_names.append(str(iterator))

sheet_ans_2 = ipysheet.sheet(rows=(s+1)*q, columns=8,column_headers=col_names,row_headers=row_names,layout=Layout(height='auto', width='auto'),row_width='')
for iterator1 in range(8):
    for iterator2 in range(0,(s+1)*q,i):
        ipysheet.cell(iterator2,iterator1,0)

sheet_ans_2

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='numeric', value=0), Cell(column_…

In [89]:
df4 = ipysheet.to_dataframe(sheet_ans_2)
df4


,$e_2$,$u_3$,$e_3 = e_2 - u_3$,$f_3(u_3)$,$F^*_3(e_2)$,"$F_2(u_3,e_2)$",$F^*_3(e_3)$,$u_3(e_3)$
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0


In [90]:
col_names =['$e_1$','$u_2$','$e_2 = e_1 - u_2$','$f_2(u_2)$','$F^*_2(e_1)$','$F_1(u_2,e_1)$','$F^*_2(e_2)$','$u_2(e_2)$']
row_names = []
for iterator in range(0,(s+1)*q,i):
    row_names.append(str(iterator))

sheet_ans_3 = ipysheet.sheet(rows=(s+1)*q, columns=8,column_headers=col_names,row_headers=row_names,layout=Layout(height='auto', width='auto'),row_width='')
for iterator1 in range(8):
    for iterator2 in range(0,(s+1)*q,i):
        ipysheet.cell(iterator2,iterator1,0)

sheet_ans_3

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='numeric', value=0), Cell(column_…

In [91]:
df5 = ipysheet.to_dataframe(sheet_ans_3)
df5

,$e_1$,$u_2$,$e_2 = e_1 - u_2$,$f_2(u_2)$,$F^*_2(e_1)$,"$F_1(u_2,e_1)$",$F^*_2(e_2)$,$u_2(e_2)$
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0


In [92]:
col_names =['$e_0$','$u_1$','$e_1 = e_0 - u_1$','$f_1(u_1)$','$F^*_1(e_0)$','$F_0(u_1,e_0)$','$F^*_1(e_1)$','$u_1(e_1)$']
row_names = []
for iterator in range(0,(s+1)*q,i):
    row_names.append(str(iterator))

sheet_ans_4 = ipysheet.sheet(rows=(s+1)*q, columns=8,column_headers=col_names,row_headers=row_names,layout=Layout(height='auto', width='auto'),row_width='')
for iterator1 in range(8):
    for iterator2 in range(0,(s+1)*q,i):
        ipysheet.cell(iterator2,iterator1,0)

sheet_ans_4

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='numeric', value=0), Cell(column_…

In [93]:
df6 = ipysheet.to_dataframe(sheet_ans_4)
df6

,$e_0$,$u_1$,$e_1 = e_0 - u_1$,$f_1(u_1)$,$F^*_1(e_0)$,"$F_0(u_1,e_0)$",$F^*_1(e_1)$,$u_1(e_1)$
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0


In [94]:
lastanswer = []
print("Количество средств на каждое предприятие")
ans = [0 for j in range(q)]

ans2 = []
for j in range(q):
    ans[j] = (widgets.Text(placeholder='Размер инвестиций в предприятие '+str(j),description='',disabled=False))
    display(ans[j])

def path_check():
    global check
    check = 1
    for j in range(q):
        ans2.append(int(ans[j].value))
    print("Задание решено верно! ")
    
    
im = widgets.interact_manual(path_check);
im.widget.children[0].description = 'Добавить'
im.widget.children[0].layout = Layout(width='30%')

Количество средств на каждое предприятие


Text(value='', placeholder='Размер инвестиций в предприятие 0')

Text(value='', placeholder='Размер инвестиций в предприятие 1')

Text(value='', placeholder='Размер инвестиций в предприятие 2')

Text(value='', placeholder='Размер инвестиций в предприятие 3')

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [95]:
print("Введите ФИО, номер группы и код доступа")
name = widgets.Text(placeholder='ФИО',description='',disabled=False)
key = widgets.Text(placeholder='Код доступа',description='',disabled=False)
display(name,key)
def send_clicked():
    key_tmp = int(key.value)
    name_tmp = name.value
    if(key_tmp==1234):
        key.value = ""
        name.value = ""
        !jupyter nbconvert --config config.py 5_Optimal_investment.ipynb
        print(name_tmp)
        print("Данные отправлены! Спасибо!")
    else:
        print("Неверный код доступа")
    
im = widgets.interact_manual(send_clicked);
im.widget.children[0].description = 'Отправить'
im.widget.children[0].layout = Layout(width='30%')

Введите ФИО, номер группы и код доступа


Text(value='', placeholder='ФИО')

Text(value='', placeholder='Код доступа')

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…